In [1]:
%cd ..

/Users/philipphager/Developer/ultr-cm-vs-ips


In [2]:
import altair as alt
#from altair_saver import save
import pandas as pd
from util import load_experiment

/Users/philipphager/.pyenv/versions/3.9.7/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
baseline_df, val_df, test_df = load_experiment("dataset_size")

In [4]:
baseline_df["model"] = "Production Ranker"

In [8]:
model2name = {
    "Neural PBM - Unbiased": "PBM - Unbiased",
    "Neural PBM - Biased": "PBM - Biased",
    "Neural PBM - Estimated bias": "PBM - Estimated Bias",
    "Pointwise IPS - Unbiased": "Point. IPS - Unbiased",
    "Pointwise IPS - Biased": "Point. IPS - Biased",
}

test_df.model = test_df.model.map(model2name)

In [9]:
def plot(dataset_df, legend=True, width=250, height=225, metric="average_relevant_position", title="", y=[0, 1.0], clip=False, label_y=True): 
    lines = alt.Chart(dataset_df, width=width, height=height, title=title).mark_line(clip=clip).encode(
        x=alt.X("n_sessions", scale=alt.Scale(type="log"), title="Number of Train Queries", axis=alt.Axis(format="~s")),
        y=alt.Y(f"mean({metric})", scale=alt.Scale(zero=False, domain=y), title=metric if label_y else None),
        color=alt.Color("model", legend=None),
        tooltip=[f"count({metric})", "n_sessions", f"mean({metric})"]
    )

    marks = alt.Chart(dataset_df).mark_point(clip=clip, size=50).encode(
        x=alt.X("n_sessions", scale=alt.Scale(type="log"), title="Number of Train Queries", axis=alt.Axis(format="~s")),
        y=alt.Y(f"mean({metric})", scale=alt.Scale(zero=False)),
        shape=alt.Shape("model"),
        color=alt.Color("model", legend=None),
        tooltip=[f"count({metric})", "n_sessions", f"mean({metric})"]
    )

    ci = alt.Chart(dataset_df).mark_errorband(opacity=0.5, clip=clip).encode(
        x=alt.X("n_sessions", scale=alt.Scale(type="log"), title="Number of Train Queries", axis=alt.Axis(format="~s")),
        y=alt.Y(metric, scale=alt.Scale(zero=False)),
        color=alt.Color("model", legend=None),
    )

    return alt.layer(
        lines,
        marks,
        ci
    )

yahoo_df = pd.concat([test_df[test_df.dataset == "Yahoo"], baseline_df[baseline_df.dataset == "Yahoo"]])
istella_df = pd.concat([test_df[test_df.dataset == "Istella-S"], baseline_df[baseline_df.dataset == "Istella-S"]])
mslr_df = pd.concat([test_df[test_df.dataset == "MSLR-Web30K"], baseline_df[baseline_df.dataset == "MSLR-Web30K"]])

chart = (
    plot(mslr_df, legend=False, metric="nDCG@10", title="MSLR-WEB30K", y=[0.2, .5], clip=True, label_y=True) |
    plot(istella_df, legend=False, metric="nDCG@10", title="Istella", y=[0.5, 0.75], clip=True, label_y=False) |
    plot(yahoo_df, legend=True, metric="nDCG@10", title="Yahoo", y=[0.59, 0.75], clip=True, label_y=False)
).configure_legend(
    orient="bottom",
    title=None,
    labelFont="serif",
    labelFontSize=14,
    columnPadding=20,
).configure_title(
    fontSize=14,
    fontWeight="normal",
    font="serif"
).configure_axis(
    titlePadding=10,
    titleFontSize=14,
    titleFontWeight="normal",
    titleFont="serif",
    labelFontSize=10,
    labelFontWeight="normal",
    labelFont="serif",
    tickCount=6
)

#save(chart, "figures/results.pdf")
chart

alt.HConcatChart(...)

In [10]:
mslr_df.groupby(["model", "n_sessions"]).count()

ARP  test_ndcg@1  nDCG@5  nDCG@10  nDCG  \
model                 n_sessions                                            
Point. IPS - Unbiased 100          10           10      10       10    10   
                      1000         10           10      10       10    10   
                      10000        10           10      10       10    10   
                      100000       10           10      10       10    10   
                      1000000      10           10      10       10    10   
                      10000000      6            6       6        6     6   
Production Ranker     100          10           10      10       10    10   
                      1000         10           10      10       10    10   
                      10000        10           10      10       10    10   
                      100000       10           10      10       10    10   
                      1000000      10           10      10       10    10   
                      10000000      7            7       7        7     7   

                                  epoch  dir  random_state  experiment_name  \
model                 n_sessions                                              
Point. IPS - Unbiased 100            10   10            10               10   
                      1000           10   10            10               10   
                      10000          10   10            10               10   
                      100000         10   10            10               10   
                      1000000        10   10            10               10   
                      10000000        6    6             6                6   
Production Ranker     100             0    0            10               10   
                      1000            0    0            10               10   
                      10000           0    0            10               10   
                      100000          0    0            10               10   
                      1000000         0    0            10               10   
                      10000000        0    0             7                7   

                                  trainer__target_  ...  model__target_  \
model                 n_sessions                    ...                   
Point. IPS - Unbiased 100                       10  ...              10   
                      1000                      10  ...              10   
                      10000                     10  ...              10   
                      100000                    10  ...              10   
                      1000000                   10  ...              10   
                      10000000                   6  ...               6   
Production Ranker     100                       10  ...              10   
                      1000                      10  ...              10   
                      10000                     10  ...              10   
                      100000                    10  ...              10   
                      1000000                   10  ...              10   
                      10000000                   7  ...               7   

                                  model_layers  model_dropouts  \
model                 n_sessions                                 
Point. IPS - Unbiased 100                   10              10   
                      1000                  10              10   
                      10000                 10              10   
                      100000                10              10   
                      1000000               10              10   
                      10000000               6               6   
Production Ranker     100                   10              10   
                      1000                  10              10   
                      10000                 10              10   
                      100000                10              10   
     

# altair_saver-0.5.0
#!pip install altair_saver
# !pip install selenium
!conda install -c conda-forge vega-cli vega-lite-cli -y
# altair_saver